In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
import os
import re
import string
from string import digits
from keras.models import model_from_json
import torch
from transformers import BertTokenizer, BertModel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 7.2 MB/s 
     |████████████████████████████████| 7.6 MB 93.0 MB/s 
     |████████████████████████████████| 182 kB 66.6 MB/s 


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/MINI PROJECT/marathi_test_without_labels.csv')
test_df

,id,tweet
0,0,पूर्व लडाखमधील मुखपरी येथे प्रत्यक्ष ताबारेषेज...
1,1,कोणत्याही रिलेशनशिप मध्ये सुंदर दिसणं खूप महत...
2,2,भारत ऑगस्ट ला स्वतंत्र झाला आणि त्यानंतर तब्...
3,3,स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यास...
4,4,व्या नंबरची अर्थव्यवस्था आहे भारताची जगात पर्...
...,...,...
505,505,चायला हा मराठीत कधी पासून ट्विट करायला लागला ...
506,506,मदत तातडीने द्यायला हवी महिने अधिकारी गोट्या ...
507,507,@USER @USER @USER @USER @USER रंगा बिल्ला ने श...
508,508,@USER काही लोकं अजूनही म्हणतात की पूर आल्यावर ...


### **Preprocessing the test dataframe**

In [ ]:
#replacing " ' " with an emptyspace
test_df['tweet'] = [re.sub("'",'',str(x)) for x in test_df['tweet']]

#removing the punctuations
def remove_punc(text_list):
  table = str.maketrans('', '', string.punctuation)
  removed_punc_text = []
  for sent in text_list:
    sentance = [w.translate(table) for w in sent.split(' ')]
    removed_punc_text.append(' '.join(sentance))
  return removed_punc_text

test_df['tweet'] = remove_punc(test_df['tweet'])

# removing the digits 
test_df['tweet'] = [re.sub("[२३०८१५७९४६]","",x) for x in test_df['tweet']]
#removing emojis from the sentence
test_df['tweet'] = [re.sub("[\u200d]","",x) for x in test_df['tweet']]


# removing the starting and ending whitespaces
test_df['tweet'] = [x.strip() for x in test_df['tweet']]

In [ ]:
test_df

,id,tweet
0,0,पूर्व लडाखमधील मुखपरी येथे प्रत्यक्ष ताबारेषेज...
1,1,कोणत्याही रिलेशनशिप मध्ये सुंदर दिसणं खूप महत...
2,2,भारत ऑगस्ट ला स्वतंत्र झाला आणि त्यानंतर तब्...
3,3,स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यास...
4,4,व्या नंबरची अर्थव्यवस्था आहे भारताची जगात पर्...
...,...,...
505,505,चायला हा मराठीत कधी पासून ट्विट करायला लागला ...
506,506,मदत तातडीने द्यायला हवी महिने अधिकारी गोट्या ...
507,507,USER USER USER USER USER रंगा बिल्ला ने शिवसेन...
508,508,USER काही लोकं अजूनही म्हणतात की पूर आल्यावर क...


### **Loading Mbert**

In [ ]:
tokenizers = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
mbert_model = BertModel.from_pretrained("bert-base-multilingual-uncased")

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### **Testing func.**

In [ ]:
import time

In [ ]:
def embed_text(text):
  a=[]
  encoded_input = tokenizers(text, return_tensors='pt')
  output = mbert_model(**encoded_input)
  a.append(output.last_hidden_state[0][0].detach().numpy())
  a=np.asarray(a)
  b=a.reshape(a.shape[0],-1,a.shape[1])
  return b
def prediction_A(tweet,modelA):
  map_target_A={0:"NOT",1:"OFF"}
  tweet_embeddings=embed_text(tweet)
  output_a=np.argmax(modelA.predict(tweet_embeddings))
  output_a=map_target_A[output_a]
  return output_a

def prediction_B(tweet,modelB):
  map_target_B={0:"UNT",1:"TIN"}
  tweet_embeddings=embed_text(tweet)
  output_b=np.argmax(modelB.predict(tweet_embeddings))
  output_b=map_target_B[output_b]
  return output_b

def prediction_C(tweet,modelC):
  map_target_C={0:"IND",1:"GRP",2:"OTH"}
  tweet_embeddings=embed_text(tweet)
  output_c=np.argmax(modelC.predict(tweet_embeddings))
  output_c=map_target_C[output_c]
  return output_c



def model_test(tweet,modelA,modelB,modelC):

  output_a=prediction_A(tweet,modelA)
  time.sleep(2)
  if(6=="NOT"):
    return output_a,"NOT","NOT"
  else:
    output_b=prediction_B(tweet,modelB)
    time.sleep(2)
    if(output_b=='UNT'):
      return output_a,output_b,output_b
    elif(output_b=='TIN'):
      output_c=prediction_C(tweet,modelC)
      time.sleep(2)
      return output_a,output_b,output_c


### **Loading the models**

In [ ]:
# load json and create model
file = open('/content/drive/MyDrive/MINI PROJECT/model_weights/mbert_BiLSTM_subtaskA.json', 'r')
modelA_json = file.read()
file.close()
modelA = model_from_json(modelA_json)
# load weights
modelA.load_weights('/content/drive/MyDrive/MINI PROJECT/model_weights/mbert_BiLSTM_subtaskA.h5')

In [ ]:
# load json and create model
file = open('/content/drive/MyDrive/MINI PROJECT/model_weights/subtaskB.json', 'r')
modelB_json = file.read()
file.close()
modelB = model_from_json(modelB_json)
# load weights
modelB.load_weights('/content/drive/MyDrive/MINI PROJECT/model_weights/subtaskB.h5')

In [ ]:
# load json and create model
file = open('/content/drive/MyDrive/MINI PROJECT/model_weights/subtaskC.json', 'r')
modelC_json = file.read()
file.close()
modelC = model_from_json(modelC_json)
# load weights
modelC.load_weights('/content/drive/MyDrive/MINI PROJECT/model_weights/subtaskC.h5')

### **Testing on the single sentences**

In [ ]:
text="महाराष्ट्रातील जनतेला व्हेंटिलेटर मास्क पीपीई किट देणार नाहीत पण एका माथेफिरू बाईला  ग्रेड सुरक्षा देणार"
print(text)
a,b,c=model_test(text,modelA,modelB,modelC)
a,b,c

महाराष्ट्रातील जनतेला व्हेंटिलेटर मास्क पीपीई किट देणार नाहीत पण एका माथेफिरू बाईला  ग्रेड सुरक्षा देणार
1/1 [==============================] - 0s 121ms/step


('OFF', 'TIN', 'OTH')

In [ ]:
text=test_df['tweet'][0]
print(text)
a,b,c=model_test(text,modelA,modelB,modelC)
a,b,c

पूर्व लडाखमधील मुखपरी येथे प्रत्यक्ष ताबारेषेजवळ सोमवारी जमलेले चिनी सैन्य भाले लोखंडी शिगा आ…
1/1 [==============================] - 0s 20ms/step


('NOT', 'NOT', 'NOT')

In [ ]:
text=test_df['tweet'][509]
print(text)
a,b,c=model_test(text,modelA,modelB,modelC)
a,b,c

USER सगळ्यात जास्त वास मारणारी पुच्ची
1/1 [==============================] - 0s 16ms/step


('OFF', 'UNT', 'UNT')

In [ ]:
import time

### **Testing on the test dataframe**

In [ ]:
subtask_a=[]
subtask_b=[]
subtask_c=[]

for i in test_df['tweet']:
  
  a,b,c=model_test(i,modelA,modelB,modelC)
  subtask_a.append(a)
  subtask_b.append(b)
  subtask_c.append(c)


test_df['subtask_a']=subtask_a
test_df['subtask_b']=subtask_b
test_df['subtask_c']=subtask_c

1/1 [==============================] - 0s 15ms/step


In [ ]:
test_df

,id,tweet,subtask_a,subtask_b,subtask_c
0,0,पूर्व लडाखमधील मुखपरी येथे प्रत्यक्ष ताबारेषेज...,NOT,NOT,NOT
1,1,कोणत्याही रिलेशनशिप मध्ये सुंदर दिसणं खूप महत...,NOT,NOT,NOT
2,2,भारत ऑगस्ट ला स्वतंत्र झाला आणि त्यानंतर तब्...,NOT,NOT,NOT
3,3,स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यास...,NOT,NOT,NOT
4,4,व्या नंबरची अर्थव्यवस्था आहे भारताची जगात पर्...,NOT,NOT,NOT
...,...,...,...,...,...
505,505,चायला हा मराठीत कधी पासून ट्विट करायला लागला ...,OFF,UNT,UNT
506,506,मदत तातडीने द्यायला हवी महिने अधिकारी गोट्या ...,OFF,UNT,UNT
507,507,USER USER USER USER USER रंगा बिल्ला ने शिवसेन...,OFF,TIN,OTH
508,508,USER काही लोकं अजूनही म्हणतात की पूर आल्यावर क...,OFF,UNT,UNT


In [ ]:
test_df.to_csv('predicted_Test_DF.csv')

In [ ]:
    # def model_test(tweet,modelA,modelB,modelC):

#   output_a=prediction_A(tweet,modelA)
#   if(output_a=="NOT"):
#     return output_a,np.nan,np.nan
#   else:
#     output_b=prediction_B(tweet,modelB)
#     if(output_b=='NOT'):
#       return output_a,np.nan,np.nan
#     elif(output_b=="UNT"):
#       return output_a,output_b,np.nan
#     elif(output_b=="TIN"):
#       output_c=prediction_C(tweet,modelC)
#       if(output_c=='NOT'):
#         return output_a,np.nan,np.nan
#       elif(output_c=="UNT"):
#         return output_a,"UNT",np.nan
#       else:
#         return output_a,output_b,output_c

    # if(output_b=='NOT'):
    #   return output_a,output_b,"NOT"
    # elif(output_b=="UNT"):
    #   return output_a,output_b,"UNT"
    # else:
    #   output_c=prediction_C(tweet,modelC)
    #   if(output_c=='NOT'):
    #     return output_a,"NOT",output_c
    #   elif(output_c=="UNT"):
    #     return output_a,"UNT","UNT"
    #   else:
    #     return output_a,output_b,output_c

# def prediction(tweet,modelA,modelB,modelC):
#   map_target_A={0:"NOT",1:"OFF"}
#   map_target_B={0:"NOT",1:"UNT",2:"TIN"}
#   map_target_C={0:"NOT",1:"UNT",2:"IND",3:"GRP",4:"OTH"}

#   tweet_embeddings=embed_text(tweet)
#   output_a=np.argmax(modelA.predict(tweet_embeddings))
#   output_a=map_target_A[output_a]
#   subtask_a=output_a
#   if(output_a=="OFF"):
#     output_b=np.argmax(modelB.predict(tweet_embeddings))
#     output_b=map_target_B[output_b]
#     subtask_b=output_b
#     if(output_b=="TIN"):
#       output_c=np.argmax(modelC.predict(tweet_embeddings))
#       output_c=map_target_C[output_c]
#       subtask_c=output_c
#     elif(output_b=="UNT"):
#       subtask_c="UNT"
#     elif(output_b=="NOT"):
#       subtask_c="NOT"
#   else:
#     subtask_b="NOT"
#     subtask_c="NOT"

#   return subtask_a,subtask_b,subtask_c